In [1]:
import pandas as pd
import csv 
import geopandas as gpd
from datetime import timedelta
from shapely.geometry import Point
from shapely import wkt
import glob

In [22]:
#read csv files and adjust to datetime
C_im_date = pd.read_csv('../data/Cienega/CienegaImageryDates.csv', parse_dates=['date'])
C_sur_date = pd.read_csv('../data/Cienega/Cienega_survey_dates.csv', parse_dates=['Cienega date'])
C_sur_date['Cienega date'] = pd.to_datetime(C_sur_date['Cienega date'])
C_im_date['date'] = pd.to_datetime(C_im_date['date'])

C_hyd = pd.read_csv('../data/Cienega/CienegaHydroData.csv')
C_hyd['datetime'] = pd.to_datetime(C_hyd['datetime'])

C_sur_date = C_sur_date.dropna(subset=['Cienega date'])
C_im_date = C_im_date.dropna(subset=['date'])

C_surveyData = pd.read_csv('../data/Cienega/Cienega_surveyData.csv')


In [3]:
#Read processed imagery in a loop

In [4]:
#find matching dates between survey and imagery
matching_dates = []
tolerance = timedelta(days = 5)


for date1 in C_sur_date['Cienega date']:
    exact_date = False
    tol = False 
    for date2 in C_im_date['date']:
        if date1 == date2:
            matching_dates.append({'Survey': date1, 'Imagery': date2})
            exact_date = True
    if not exact_date:
        for date2 in C_im_date['date']:
            if abs(date1 - date2) <= tolerance:
                matching_dates.append({'Survey': date1, 'Imagery': date2})
                tol = True
        if not tol:
            for date2 in C_im_date['date']:
                if abs(date1-date2) < timedelta(days = 10): 
                    matching_dates.append({'Survey': date1, 'Imagery': date2})


matching_dates_df = pd.DataFrame(matching_dates)



In [5]:
C_datessurData = pd.merge(matching_dates_df, C_hyd, left_on = 'Survey', right_on = 'datetime', how = 'left')
C_datesimData = pd.merge(matching_dates_df, C_hyd, left_on = 'Imagery', right_on = 'datetime')
C_datessurData = C_datessurData.drop(columns = ['Imagery','datetime'])
C_datesimData = C_datesimData.drop(columns = ['Survey','datetime'])

In [6]:
#sum precipitation for dates in between survey and imagery
def sum_pdatesbetween(d1, d2):
    r = pd.date_range(start=min(d1,d2), end=max(d1,d2))
    return C_hyd[C_hyd['datetime'].isin(r)]['P [mm]'].sum()

In [7]:
#making a dataframe to determine which imagery dates to use
Ch = pd.DataFrame([])

Ch['Survey'] = matching_dates_df['Survey']
Ch['Imagery'] = matching_dates_df['Imagery']
Ch['sum_P'] = [sum_pdatesbetween(C_datessurData.loc[i, 'Survey'], C_datesimData.loc[i, 'Imagery']) for i in range(len(Ch))]
Ch['Q_diff [%]'] = (C_datessurData['Q [mm/d]'] - C_datesimData['Q [mm/d]']) / C_datessurData['Q [mm/d]'] * 100
Ch['Use/not'] = ['use', 'use', 'use', 'use', 'not', 'not', 'use?',
                 'not', 'only option', 'not', 'not', 'use', 'not',
                 'not', 'not', 'use', 'not', 'not', 'not', 'not',
                 'use', 'use', 'use', 'only option','use', 'use', 
                 'use','use', 'not', 'not', 'not', 'not', 'use', 
                 'not', 'use', 'use', 'use', 'use', 'use', 'not', 
                 'not', 'not', 'not', 'use', 'not', 'not', 'not', 
                 'not', 'not', 'use', 'not', 'not', 'not', 'use', 
                 'not', 'use', 'use']


conditions = (Ch['sum_P'] > 3) | (Ch['Q_diff [%]'] > 8) | (Ch['Use/not'] == 'not')

Ch = Ch[~conditions]

Ch = Ch.drop(columns=['Use/not'])

Ch

,Survey,Imagery,sum_P,Q_diff [%]
0,2016-09-23,2016-09-22,0.0,0.000000
1,2017-03-16,2017-03-13,0.0,0.000000
2,2017-06-09,2017-06-09,0.0,0.000000
3,2017-09-19,2017-09-21,0.0,4.587156
6,2017-12-08,2017-12-07,0.0,-2.803738
11,2018-06-05,2018-06-04,0.0,7.142857
15,2018-09-08,2018-09-07,0.0,1.785714
20,2018-12-17,2018-12-17,0.0,0.000000
21,2019-03-26,2019-03-26,0.0,0.000000
22,2019-06-07,2019-06-07,0.0,0.000000


In [8]:
Ch.to_csv('../data/Cienega/Cienega_survey_imagery_HydroData.csv', encoding='utf-8', index=False)

In [13]:
#reading surveydata, making it into a geodataframe and adding x and y from the geometry to facilitate merge
#C_surveyData['geometry'] = C_surveyData['geometry'].apply(wkt.loads)
gdf = gpd.GeoDataFrame(C_surveyData, geometry = 'geometry', crs='EPSG:26912')

gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y
gdf = gdf[['geometry', 'x', 'y', 'Year', 'wetdry']]
gdf['Year'] = pd.to_datetime(gdf['Year'])
gdf

,geometry,x,y,Year,wetdry
0,POINT (533719.252 3542427.373),533719.252411,3.542427e+06,2004-06-25,dry
1,POINT (533719.787 3542422.402),533719.786671,3.542422e+06,2004-06-25,dry
2,POINT (533720.321 3542417.431),533720.320931,3.542417e+06,2004-06-25,dry
3,POINT (533720.855 3542412.459),533720.855191,3.542412e+06,2004-06-25,dry
4,POINT (533721.389 3542407.488),533721.389451,3.542407e+06,2004-06-25,dry
...,...,...,...,...,...
237685,POINT (533717.227 3542446.217),533717.227336,3.542446e+06,2013-09-05,dry
237686,POINT (533717.762 3542441.246),533717.761591,3.542441e+06,2013-09-05,dry
237687,POINT (533718.296 3542436.274),533718.295846,3.542436e+06,2013-09-05,dry
237688,POINT (533718.830 3542431.303),533718.830102,3.542431e+06,2013-09-05,dry


In [21]:
C_mergedata = Ch.merge(gdf, left_on = 'Survey', right_on = 'Year')
C_mergedata = C_mergedata.drop(columns=['Year', 'Survey', 'sum_P', 'Q_diff [%]'])
#split on wet/dry
C_mergewet = C_mergedata[~(C_mergedata['wetdry'] == 'dry')]
C_mergedry = C_mergedata[~(C_mergedata['wetdry'] == 'wet')]

In [17]:
#reading and concatenating the processed imagery 
path = '../data/Cienega/processed_imagery'

processed_imagery = glob.glob(path + '/*.csv')
processed_imagery.sort(key = lambda x: int(x.split('_buffer_')[1].split('.')[0]))
con_ready_imagery = []
for processed in processed_imagery:
    df= pd.read_csv(processed)
    con_ready_imagery.append(df)

concatenated = pd.concat(con_ready_imagery)

In [19]:
concatenated['geometry'] = concatenated['geometry'].apply(wkt.loads)
gdf2 = gpd.GeoDataFrame(concatenated, geometry = 'geometry', crs='EPSG:26912')

In [20]:
gdf2

,blue,green,red,NIR,missing,NDWI,p,date,geometry
0,866.78,1160.78,1527.67,2449.56,0,-0.36,0,20181121,POINT (533719.252 3542427.373)
1,869.89,1168.11,1497.22,2404.89,0,-0.35,0,20181121,POINT (533719.787 3542422.402)
2,893.56,1189.44,1491.22,2383.44,0,-0.33,0,20181121,POINT (533720.321 3542417.431)
3,939.78,1227.22,1530.89,2385.56,0,-0.32,0,20181121,POINT (533720.855 3542412.459)
4,1072.67,1341.00,1730.22,2272.00,0,-0.26,0,20181121,POINT (533721.389 3542407.488)
...,...,...,...,...,...,...,...,...,...
232300,634.67,998.44,1149.78,3126.22,0,-0.52,0,20230701,POINT (533717.227 3542446.217)
232301,666.89,1041.00,1218.00,3116.00,0,-0.50,0,20230701,POINT (533717.762 3542441.246)
232302,806.11,1202.67,1484.00,3275.33,0,-0.46,0,20230701,POINT (533718.296 3542436.274)
232303,871.67,1294.00,1620.00,3227.22,0,-0.43,0,20230701,POINT (533718.830 3542431.303)
